In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random

# Your specific partition-node mapping
partition_nodes = {
    '30mins': ['z1014', 'z1016', 'z1017', 'z1018', 'z1019', 'z1020', 'z1021', 'z1022', 
               'z1023', 'z1024', 'z1025', 'z1026', 'z1027', 'z1028', 'z1029', 'z1032', 
               'z1033', 'z1034', 'z1035', 'z1036', 'z1038', 'z1039', 'z1040', 'z1041', 
               'z1042', 'z1043', 'z1044', 'z1045', 'z1047', 'z1048', 'z1049', 'z1050'],
    '4hours': ['z1014', 'z1016', 'z1017', 'z1018', 'z1019', 'z1020', 'z1032', 'z1033', 
               'z1034', 'z1035', 'z1036', 'z1038', 'z1039', 'z1040'],
    '12hours': ['z1021', 'z1022', 'z1023', 'z1024', 'z1025', 'z1026', 'z1027', 'z1028', 
                'z1029', 'z1041', 'z1042', 'z1043', 'z1044', 'z1045', 'z1047', 'z1048', 
                'z1049', 'z1050', 'z1051', 'z1052', 'z1053'],
    '5days': ['z1014', 'z1016', 'z1017', 'z1018', 'z1019', 'z1020', 'z1041', 'z1042', 
              'z1043', 'z1044', 'z1045', 'z1047', 'z1048', 'z1049', 'z1050', 'z1051', 
              'z1052', 'z1053'],
    'gpu': ['z018', 'z019']
}

all_nodes = set()
for nodes in partition_nodes.values():
    all_nodes.update(nodes)
all_nodes = list(all_nodes)

node_partitions = {node: [] for node in all_nodes}
for part, nodes in partition_nodes.items():
    for node in nodes:
        node_partitions[node].append(part)

node_usage = {node: random.uniform(0.1, 0.9) for node in all_nodes}
partition_usage = {part: random.uniform(0.1, 0.9) for part in partition_nodes}

G = nx.Graph()

for part in partition_nodes.keys():
    G.add_node(part, size=1200, type='partition', 
               color=plt.cm.YlOrRd(partition_usage[part]))

for node in all_nodes:
    G.add_node(node, size=200, type='node', 
               color=plt.cm.YlOrRd(node_usage[node]))
    
    for part in node_partitions[node]:
        G.add_edge(node, part)

plt.figure(figsize=(14, 12))

partition_pos = nx.circular_layout(list(partition_nodes.keys()))

pos = nx.spring_layout(G, k=0.3, iterations=50, 
                     fixed=list(partition_nodes.keys()),
                     pos={p: partition_pos[p] for p in partition_nodes.keys()})

partition_nodes_list = list(partition_nodes.keys())
compute_nodes_list = all_nodes

# Draw partition nodes (larger squares)
partition_colors = [G.nodes[node]['color'] for node in partition_nodes_list]
nx.draw_networkx_nodes(G, pos, nodelist=partition_nodes_list, 
                     node_size=[G.nodes[node]['size'] for node in partition_nodes_list],
                     node_color=partition_colors, alpha=0.8, 
                     node_shape='s')

# Draw compute nodes (smaller circles)
node_colors = [G.nodes[node]['color'] for node in compute_nodes_list]
nx.draw_networkx_nodes(G, pos, nodelist=compute_nodes_list, 
                     node_size=[G.nodes[node]['size'] for node in compute_nodes_list],
                     node_color=node_colors, alpha=0.8)

# Draw edges with color based on partition
for part in partition_nodes_list:
    edges = [(part, node) for node in partition_nodes[part]]
    nx.draw_networkx_edges(G, pos, edgelist=edges, width=1.0, alpha=0.5,
                         edge_color=G.nodes[part]['color'])

# Draw labels with different sizes for partitions and nodes
partition_labels = {node: node for node in partition_nodes_list}
nx.draw_networkx_labels(G, pos, labels=partition_labels, 
                      font_size=14, font_weight='bold')

# For nodes, only label a subset to avoid clutter - shared nodes
shared_nodes = [node for node in all_nodes if len(node_partitions[node]) > 1]
node_labels = {node: node for node in shared_nodes}
nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=8)

# Add a color bar to show resource usage scale
sm = plt.cm.ScalarMappable(cmap=plt.cm.YlOrRd, norm=plt.Normalize(0, 1))
sm.set_array([])
cbar = plt.colorbar(sm, ax=plt.gca().axes)
cbar.set_label('Resource Usage (%)')

# Add a legend for node types
plt.scatter([], [], s=200, c='gray', alpha=0.5, marker='s', label='Partition')
plt.scatter([], [], s=200, c='gray', alpha=0.5, marker='o', label='Node')
plt.legend(scatterpoints=1, frameon=False, labelspacing=1)

plt.title('SLURM Partition and Node Resource Usage')
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
import plotly.graph_objects as go
import networkx as nx
import numpy as np
import random
import plotly.express as px

# Your specific partition-node mapping
partition_nodes = {
    '30mins': ['z1014', 'z1016', 'z1017', 'z1018', 'z1019', 'z1020', 'z1021', 'z1022', 
               'z1023', 'z1024', 'z1025', 'z1026', 'z1027', 'z1028', 'z1029', 'z1032', 
               'z1033', 'z1034', 'z1035', 'z1036', 'z1038', 'z1039', 'z1040', 'z1041', 
               'z1042', 'z1043', 'z1044', 'z1045', 'z1047', 'z1048', 'z1049', 'z1050'],
    '4hours': ['z1014', 'z1016', 'z1017', 'z1018', 'z1019', 'z1020', 'z1032', 'z1033', 
               'z1034', 'z1035', 'z1036', 'z1038', 'z1039', 'z1040'],
    '12hours': ['z1021', 'z1022', 'z1023', 'z1024', 'z1025', 'z1026', 'z1027', 'z1028', 
                'z1029', 'z1041', 'z1042', 'z1043', 'z1044', 'z1045', 'z1047', 'z1048', 
                'z1049', 'z1050', 'z1051', 'z1052', 'z1053'],
    '5days': ['z1014', 'z1016', 'z1017', 'z1018', 'z1019', 'z1020', 'z1041', 'z1042', 
              'z1043', 'z1044', 'z1045', 'z1047', 'z1048', 'z1049', 'z1050', 'z1051', 
              'z1052', 'z1053'],
    'gpu': ['z018', 'z019']
}

# Extract all unique nodes
all_nodes = set()
for nodes in partition_nodes.values():
    all_nodes.update(nodes)
all_nodes = list(all_nodes)

# Create reverse mapping (node to partitions)
node_partitions = {node: [] for node in all_nodes}
for part, nodes in partition_nodes.items():
    for node in nodes:
        node_partitions[node].append(part)

# Generate random usage data
node_usage = {node: random.uniform(10, 90) for node in all_nodes}
partition_usage = {}

# Calculate partition usage as average of its nodes
for part, nodes in partition_nodes.items():
    if nodes:  # Ensure partition has nodes
        partition_usage[part] = sum(node_usage[node] for node in nodes if node in node_usage) / len(nodes)
    else:
        partition_usage[part] = 0

# Create graph
G = nx.Graph()

# Add partition nodes
for part in partition_nodes.keys():
    G.add_node(part, type='partition', usage=partition_usage[part])

# Add compute nodes
for node in all_nodes:
    G.add_node(node, type='node', usage=node_usage[node])
    
    # Connect nodes to their partitions
    for part in node_partitions[node]:
        G.add_edge(node, part)

# Position the partitions in a circle
partition_pos = nx.circular_layout(list(partition_nodes.keys()), scale=10)

# Use spring layout for all nodes, fixing partition positions
pos = nx.bipartite_layout(G, list(partition_nodes.keys()))

# Prepare node data for plotting
node_x = []
node_y = []
node_text = []
node_size = []
node_color = []
node_symbols = []

for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    
    if node in partition_nodes:  # It's a partition
        usage = partition_usage[node]
        # Count nodes in partition and shared nodes
        total_nodes = len(partition_nodes[node])
        shared_nodes = sum(1 for n in partition_nodes[node] if len(node_partitions[n]) > 1)
        node_text.append(f"Partition: {node}<br>Usage: {usage:.1f}%<br>Nodes: {total_nodes}<br>Shared: {shared_nodes}")
        node_size.append(50)  # Larger size for partitions
        node_color.append(usage)  # Color based on usage
        node_symbols.append('square')  # Square for partitions
    else:  # It's a compute node
        usage = node_usage[node]
        parts = ", ".join(node_partitions[node])
        node_text.append(f"Node: {node}<br>Usage: {usage:.1f}%<br>Partitions: {parts}")
        node_size.append(20)  # Smaller size for nodes
        node_color.append(usage)  # Color based on usage
        node_symbols.append('circle')  # Circle for nodes

# Create a continuous color scale
colorscale = px.colors.sequential.YlOrRd

# This approach creates separate traces for each edge to avoid the list color error
edge_traces = []

# Create separate edge traces for each partition
for part in partition_nodes:
    part_color = colorscale[int(partition_usage[part]/100 * (len(colorscale)-1))]
    
    for node in partition_nodes[part]:
        if node in pos:  # Make sure the node exists in our position dictionary
            # Create edge between partition and node
            x0, y0 = pos[part]
            x1, y1 = pos[node]
            
            edge_trace = go.Scatter(
                x=[x0, x1, None],
                y=[y0, y1, None],
                mode='lines',
                line=dict(width=0.5, color=part_color),
                hoverinfo='none',
                showlegend=False
            )
            
            edge_traces.append(edge_trace)

# Create node trace for partitions
partition_indices = [i for i, symbol in enumerate(node_symbols) if symbol == 'square']
partition_trace = go.Scatter(
    x=[node_x[i] for i in partition_indices],
    y=[node_y[i] for i in partition_indices],
    mode='markers+text',
    text=[node_text[i].split('<br>')[0].split(': ')[1] for i in partition_indices],
    textposition="top center",
    hovertext=[node_text[i] for i in partition_indices],
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale=colorscale,
        color=[node_color[i] for i in partition_indices],
        size=[node_size[i] for i in partition_indices],
        colorbar=dict(
            title="Usage %",
            thickness=15,
        ),
        symbol='square',
        line=dict(color='rgba(50, 50, 50, 0.8)', width=1)
    )
)

# Create node trace for compute nodes
node_indices = [i for i, symbol in enumerate(node_symbols) if symbol == 'circle']
node_trace = go.Scatter(
    x=[node_x[i] for i in node_indices],
    y=[node_y[i] for i in node_indices],
    mode='markers',
    hovertext=[node_text[i] for i in node_indices],
    hoverinfo='text',
    marker=dict(
        colorscale=colorscale,
        color=[node_color[i] for i in node_indices],
        size=[node_size[i] for i in node_indices],
        symbol='circle',
        line=dict(color='rgba(50, 50, 50, 0.8)', width=0.5)
    )
)

# Combine all traces
all_traces = edge_traces + [node_trace, partition_trace]

# Create the figure
fig = go.Figure(data=all_traces,
               layout=go.Layout(
                   title='SLURM Partition Resource Usage',
                   showlegend=False,
                   hovermode='closest',
                   margin=dict(b=20,l=5,r=5,t=40),
                   xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                   yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                   plot_bgcolor='rgba(240,240,240,0.8)',
                   paper_bgcolor='rgba(255,255,255,0.9)',
                   width=800,
                   height=800,
                   annotations=[
                       dict(
                           text='Node size: CPU cores • Node color: Usage %',
                           showarrow=False,
                           xref="paper", yref="paper",
                           x=0.01, y=0.01
                       )
                   ]
               )
              )

# Add some hints for using the visualization
fig.add_annotation(
    text="Hover over nodes and partitions to see details<br>Click and drag to rotate<br>Scroll to zoom",
    xref="paper", yref="paper",
    x=0.5, y=0.02,
    showarrow=False,
    font=dict(size=10)
)

# Add floating animation
frames = []
n_frames = 20

for i in range(n_frames):
    partition_jitter = 0.04
    node_jitter = 0.08
    
    # Create new positions with jitter
    new_positions = {}
    for node in G.nodes():
        x, y = pos[node]
        jitter = partition_jitter if node in partition_nodes else node_jitter
        new_positions[node] = (
            x + random.uniform(-jitter, jitter),
            y + random.uniform(-jitter, jitter)
        )
    
    # Create new traces for this frame
    frame_edge_traces = []
    
    # Create edges for this frame
    for part in partition_nodes:
        part_color = colorscale[int(partition_usage[part]/100 * (len(colorscale)-1))]
        
        for node in partition_nodes[part]:
            if node in new_positions:
                # Create edge between partition and node
                x0, y0 = new_positions[part]
                x1, y1 = new_positions[node]
                
                edge_trace = go.Scatter(
                    x=[x0, x1, None],
                    y=[y0, y1, None],
                    mode='lines',
                    line=dict(width=0.5, color=part_color),
                    hoverinfo='none'
                )
                
                frame_edge_traces.append(edge_trace)
    
    # Create node traces for this frame
    partition_x = [new_positions[part][0] for part in partition_nodes]
    partition_y = [new_positions[part][1] for part in partition_nodes]
    
    frame_partition_trace = go.Scatter(
        x=partition_x,
        y=partition_y,
        mode='markers+text',
        text=[part for part in partition_nodes],
        textposition="top center",
        hoverinfo='text',
        marker=dict(
            colorscale=colorscale,
            color=[partition_usage[part] for part in partition_nodes],
            size=[50] * len(partition_nodes),
            symbol='square'
        )
    )
    
    node_x = [new_positions[node][0] for node in all_nodes if node in new_positions]
    node_y = [new_positions[node][1] for node in all_nodes if node in new_positions]
    
    frame_node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            colorscale=colorscale,
            color=[node_usage[node] for node in all_nodes if node in new_positions],
            size=[20] * len(node_x),
            symbol='circle'
        )
    )
    
    # Add all traces to this frame
    frame_data = frame_edge_traces + [frame_node_trace, frame_partition_trace]
    frames.append(go.Frame(data=frame_data))

# Add frames to figure
fig.frames = frames

# Add animation controls
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[None, {"frame": {"duration": 100, "redraw": True}, "fromcurrent": True}]
                ),
                dict(
                    label="Pause",
                    method="animate",
                    args=[[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}]
                )
            ],
            direction="left",
            pad={"r": 10, "t": 10},
            showactive=False,
            x=0.1,
            xanchor="right",
            y=0,
            yanchor="top"
        )
    ]
)

fig.show()

TypeError: unhashable type: 'numpy.ndarray'